In [ ]:
import os

%load_ext autoreload
%autoreload 2

"""

DEcoding shapes, in pool where there are novel and learned shapes
Questions:
- Decoding for learned shapes is better
- Learned genearlize to themselves and novel to themselves more.
"""

# Load a dataset

To load and plot a dataset of neural activity across population, in a PopAnal class object.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from neuralmonkey.classes.population_mult import load_handsaved_wrapper, dfpa_match_chans_across_pa_each_bregion
from neuralmonkey.classes.population_mult import extract_single_pa


In [ ]:
# Method 1 - load a single DFallPA
DFallpa = load_handsaved_wrapper()


In [ ]:
# Method 1 - load a single DFallPA
animal = "Diego"
date = 240522
DFallpa = load_handsaved_wrapper(animal=animal, date=date, version="trial")


In [ ]:
dflab = DFallpa["pa"].values[0].Xlabels["trials"]
dflab["aborted"].value_counts()

In [ ]:
# # Method 2 - Combine two dfallpa
# DFallpa1 = load_handsaved_wrapper(animal="Diego", date=230630, version="trial")
# DFallpa2 = load_handsaved_wrapper(animal="Diego", date=230630, version="stroke")
# DFallpa = pd.concat([DFallpa1, DFallpa2]).reset_index(drop=True)


In [ ]:
# # Method 2 - Combine two dfallpa
# animal = "Diego"
# date = 231211
# COMBINE_AREAS = True

# DFallpa1 = load_handsaved_wrapper(animal=animal, date=date, version="trial", combine_areas=COMBINE_AREAS, use_time=False)
# DFallpa2 = load_handsaved_wrapper(animal=animal, date=date, version="stroke", combine_areas=COMBINE_AREAS, use_time=False)
# DFallpa = pd.concat([DFallpa1, DFallpa2]).reset_index(drop=True)


# Preprocessing

In [ ]:
from neuralmonkey.classes.population_mult import load_handsaved_wrapper, dfpa_match_chans_across_pa_each_bregion
from neuralmonkey.classes.population_mult import extract_single_pa


In [ ]:

# Prune to chans that are common across pa for each bregion (intersection of chans)|
dfpa_match_chans_across_pa_each_bregion(DFallpa)

In [ ]:
# SAVEDIR = f"/lemur2/lucas/analyses/recordings/main/decode_moment/PSYCHO_SP/{animal}-{date}"
SAVEDIR = f"/tmp/NOVEL_PRIMS/{animal}-{date}"
os.makedirs(SAVEDIR, exist_ok=True)
print(SAVEDIR)

# Preprocessing

##### Devo -- removing noisy channels

In [ ]:
from neuralmonkey.classes.population_mult import dfpa_concatbregion_preprocess_clean_bad_channels
dfpa_concatbregion_preprocess_clean_bad_channels(DFallpa, PLOT=False)

# Sqrt normalization

In [ ]:
for pa in DFallpa["pa"]:
    pa.X = pa.X**0.5

### FR Normalization

##### Method 1 - each PA normalize independently

In [ ]:
from neuralmonkey.classes.population_mult import dfallpa_preprocess_fr_normalization
# fr_normalization_method = "each_time_bin"
fr_normalization_method = "across_time_bins"
plot_savedir = "/tmp"
dfallpa_preprocess_fr_normalization(DFallpa, fr_normalization_method, plot_savedir)

##### Method 2 - Concat events (for each bregion) and normalize all same way


In [ ]:
from neuralmonkey.classes.population_mult import dfpa_concat_normalize_fr_split_multbregion
# fr_normalization_method = "each_time_bin"
fr_normalization_method = "across_time_bins"
dfpa_concat_normalize_fr_split_multbregion(DFallpa)


##### Method 3 - concat events (flexible version, only constrianed to have same n chans across PA) [works if have both trial and stroke!]

In [ ]:
from neuralmonkey.classes.population_mult import dfpa_concat_normalize_fr_split_multbregion_flex
fr_mean_subtract_method = "across_time_bins"
# fr_mean_subtract_method = "each_time_bin"
PLOT=False

pa = DFallpa["pa"].values[10]
pa.plotNeurHeat(0)

dfpa_concat_normalize_fr_split_multbregion_flex(DFallpa, fr_mean_subtract_method, PLOT)

pa = DFallpa["pa"].values[10]
pa.plotNeurHeat(0)

# Extract relevant variables


In [ ]:
from pythonlib.tools.pandastools import grouping_plot_n_samples_conjunction_heatmap
grouping_plot_n_samples_conjunction_heatmap(dflab, "seqc_0_shape", "shape_is_novel_all")

In [ ]:
from neuralmonkey.scripts.analy_pig_decode_moment_syntaxTI import dfallpa_preprocess_condition
# shape_var_suff = "shapesemgrp"|
# loc_var_suff = "loc_on_clust"
shape_var_suff = "shape"
loc_var_suff = "loc"
dfallpa_preprocess_condition(DFallpa, shape_var_suff, loc_var_suff)

# Extract beh data

In [ ]:
# from neuralmonkey.analyses.decode_moment import analy_psychoprim_prepare_beh_dataset
# DSmorphsets, map_tc_to_morph_info, map_morphset_to_basemorphinfo, map_tcmorphset_to_idxmorph, map_tcmorphset_to_info, map_morphsetidx_to_assignedbase_or_ambig, map_tc_to_morph_status = analy_psychoprim_prepare_beh_dataset(animal, date)

# Train / test

In [ ]:
from neuralmonkey.analyses.decode_moment import train_decoder_helper, pipeline_train_test_scalar_score, pipeline_train_test_scalar_score_with_splits, pipeline_train_test_scalar_score_mult_train_dataset, test_decoder_helper, train_decoder_helper_extract_train_dataset
from neuralmonkey.scripts.analy_pig_decode_moment_syntaxTI import get_dataset_params
import os

In [ ]:
# get list of base prim names

In [ ]:

SAVEDIR_BASE = "/tmp/NOVELPRIMS"
os.makedirs(SAVEDIR_BASE, exist_ok=True)

In [ ]:
date

In [ ]:
from neuralmonkey.scripts.analy_decode_moment_novelprims import analy_novelprim_prepare_dataset
analy_novelprim_prepare_dataset(DFallpa, SAVEDIR_BASE)

# One decoder for each shape - visualize generalization patterns

In [ ]:
bregion = "PMv"


In [ ]:
# Average over all train-test split
if False: # Doesnt work well, takes too long.
    cols_ignore = ["pa_idx", "decoder_idx", "train_split_idx"]
    cols_float = [col for col in dfscores.columns if dfscores[col].dtype == float]
    # cols_cat = [col for col in dfscores.columns if dfscores[col].dtype != float]
    # cols_cat = [col for col in cols_cat if col not in cols_ignore]
    cols_cat = ['decoder_class', 'pa_class', 'twind', 'epoch', 'trialcode']
    from pythonlib.tools.pandastools import aggregGeneral
    dfscores = aggregGeneral(dfscores, cols_cat, cols_float, nonnumercols="all")
    # Average over all train-test split
    [col for col in dfscores if isinstance(dfscores[col], float)]

### Method 1 -- not doing split by gridloc

In [ ]:
downsample_trials = False
if downsample_trials:
    # So that the lowest N doesnt pull all other categories down.
    n_min_per_var = 10
else:
    n_min_per_var = 7
        
TWIND_TEST = (0.05, 1.2)

do_upsample_balance=True
PLOT_DECODER = False
DO_TRAIN_TEST_SPLIT=True
PLOT_TEST_CONCATTED = True

TWIND_TRAIN = (0.05, 1.2)

# Subtrract baseline?
subtract_baseline=False
subtract_baseline_twind=None
include_null_data = False
prune_labels_exist_in_train_and_test = True

# - Train params
event_train = "03_samp"
twind_train = TWIND_TRAIN
var_train = "seqc_0_shape_pref"
filterdict_train = None

# - Test params
var_test = "seqc_0_shape_pref"
event_test = "03_samp"
which_level_test = "trial"
filterdict_test = None
# list_twind_test = [(-0.8, -0.05), TWIND_TEST]
list_twind_test = [TWIND_TEST]

# Train/test splits
do_train_splits_nsplits=10
score_user_test_data = False
PLOT_TEST_SPLIT = False

# Other params
SAVEDIR = f"{SAVEDIR_BASE}/downsample_trials={downsample_trials}-TWIND_TEST={TWIND_TEST}-dotraintestsplit={DO_TRAIN_TEST_SPLIT}"

list_bregion = DFallpa["bregion"].unique().tolist()
for bregion in list_bregion:
    savedir = f"{SAVEDIR}/{bregion}/decoder_training"
    os.makedirs(savedir, exist_ok=True)
    print(savedir)

    if not DO_TRAIN_TEST_SPLIT:
        dfscores, Dc, PAtrain, PAtest = pipeline_train_test_scalar_score(DFallpa, bregion, var_train, event_train, 
                                                                        twind_train, filterdict_train,
                                            var_test, event_test, list_twind_test, filterdict_test, savedir,
                                            include_null_data=include_null_data, decoder_method_index=None,
                                            prune_labels_exist_in_train_and_test=prune_labels_exist_in_train_and_test, PLOT=PLOT_DECODER,
                                            which_level_test=which_level_test, n_min_per_var=n_min_per_var,
                                            subtract_baseline=subtract_baseline, subtract_baseline_twind=subtract_baseline_twind,
                                            do_upsample_balance=do_upsample_balance, downsample_trials=downsample_trials)
    else:
        dfscores, dfscores_usertest, dfscores_both, decoders, trainsets, PAtest = pipeline_train_test_scalar_score_with_splits(DFallpa, 
                                                                        bregion, var_train, event_train, 
                                                                        twind_train, filterdict_train,
                                            var_test, event_test, list_twind_test, filterdict_test, savedir,
                                            include_null_data=include_null_data, decoder_method_index=None,
                                            prune_labels_exist_in_train_and_test=prune_labels_exist_in_train_and_test, 
                                            PLOT_TRAIN=PLOT_DECODER, PLOT_TEST_SPLIT=PLOT_TEST_SPLIT, PLOT_TEST_CONCATTED=PLOT_TEST_CONCATTED,
                                            which_level_test=which_level_test, n_min_per_var=n_min_per_var,
                                            subtract_baseline=subtract_baseline, subtract_baseline_twind=subtract_baseline_twind,
                                            do_upsample_balance=do_upsample_balance, downsample_trials=downsample_trials,
                                            do_train_splits_nsplits=do_train_splits_nsplits, 
                                            score_user_test_data=score_user_test_data)

    ######################## PLOTS
    from neuralmonkey.analyses.decode_moment import analy_psychoprim_score_postsamp
    savedir = f"{SAVEDIR}/{bregion}/PLOTS"
    os.makedirs(savedir, exist_ok=True)
    print(savedir)
    analy_novelprim_score_postsamp(dfscores, decoders, dflab, savedir)                

### Method 2 - splitting by gridloc

In [ ]:
from neuralmonkey.analyses.decode_moment import pipeline_train_test_scalar_score_split_gridloc
bregion = "PMv"
dflab = pa.Xlabels["trials"]
list_loc = dflab["seqc_0_loc"].unique().tolist()
auto_prune_locations = True

In [ ]:
dflab["aborted"].value_counts()

In [ ]:


        
filterdict_train = None
filterdict_test = None


### OTher params.
TWIND_TEST = (0.05, 1.2)
TWIND_TRAIN = (0.05, 1.2)

do_upsample_balance=True
PLOT_DECODER = False

# Subtrract baseline?
subtract_baseline=False
subtract_baseline_twind=None
include_null_data = False
prune_labels_exist_in_train_and_test = True

# - Train params
event_train = "03_samp"
twind_train = TWIND_TRAIN
var_train = "seqc_0_shape_pref"

# - Test params
var_test = "seqc_0_shape_pref"
event_test = "03_samp"
which_level_test = "trial"
# list_twind_test = [(-0.8, -0.05), TWIND_TEST]
list_twind_test = [TWIND_TEST]

# Other params
SAVEDIR = f"{SAVEDIR_BASE}/downsample_trials={downsample_trials}-TWIND_TEST={TWIND_TEST}-split_by_gridloc"




list_bregion = DFallpa["bregion"].unique().tolist()
for bregion in list_bregion:
    savedir_base = f"{SAVEDIR}/{bregion}/decoder_training"
    os.makedirs(savedir_base, exist_ok=True)
    print(savedir_base)

    DFSCORES, decoders, list_pa_train, list_pa_test = pipeline_train_test_scalar_score_split_gridloc(list_loc, savedir_base,
                                                                                                     DFallpa, 
                                                                    bregion, var_train, event_train, 
                                                                    twind_train, filterdict_train,
                                        var_test, event_test, list_twind_test, filterdict_test, 
                                        include_null_data=include_null_data, 
                                        prune_labels_exist_in_train_and_test=prune_labels_exist_in_train_and_test, 
                                        PLOT=PLOT_DECODER,
                                        which_level_test=which_level_test, n_min_per_var=n_min_per_var,
                                        subtract_baseline=subtract_baseline, subtract_baseline_twind=subtract_baseline_twind,
                                        do_upsample_balance=do_upsample_balance, downsample_trials=downsample_trials,
                                        auto_prune_locations=auto_prune_locations)

    # ######################## PLOTS
    # from neuralmonkey.analyses.decode_moment import analy_psychoprim_score_postsamp
    # savedir = f"{SAVEDIR}/{bregion}/PLOTS"
    # os.makedirs(savedir, exist_ok=True)
    # print(savedir)
    # analy_novelprim_score_postsamp(dfscores, decoders, dflab, savedir)                


# MULT BREGIONS - load and compare

In [ ]:
import pandas as pd
from neuralmonkey.classes.session import _REGIONS_IN_ORDER, _REGIONS_IN_ORDER_COMBINED
from pythonlib.tools.pandastools import grouping_plot_n_samples_conjunction_heatmap, stringify_values, aggregGeneral
from neuralmonkey.scripts.analy_decode_moment_psychometric import prune_dfscores_good_morphset


In [ ]:
SAVEDIR_BASE = "/lemur2/lucas/analyses/recordings/main/decode_moment/PSYCHO_SP"


In [ ]:

combine = True
downsample_trials = False
# TWIND_TEST = (0.05, 1.2)
version = 1
for animal, date in [("Pancho", 240524), ("Diego", 240523)]:
    for TWIND_TEST in [(0.05, 1.2), (0.6, 1.2)]:

        if combine:
            list_bregion = _REGIONS_IN_ORDER_COMBINED
        else:
            list_bregion = _REGIONS_IN_ORDER

        list_dfscores = []
        for bregion in list_bregion:
            path = f"{SAVEDIR_BASE}/{animal}-{date}-logistic-combine={combine}/downsample_trials={downsample_trials}-TWIND_TEST={TWIND_TEST}-version={version}/{bregion}/DFSCORES.pkl"
            print("Loading ... ", path)
            dfscores = pd.read_pickle(path)
            dfscores["bregion"] = bregion
            list_dfscores.append(dfscores)
        DFSCORES = pd.concat(list_dfscores).reset_index(drop=True)

        SAVEDIR = f"{SAVEDIR_BASE}/{animal}-{date}-logistic-combine={combine}/downsample_trials={downsample_trials}-TWIND_TEST={TWIND_TEST}-version={version}/MULT_BREGION"
        print("SAving at: ", SAVEDIR)



        # ### PReprocessing
        # assert len(DFSCORES["twind"].unique())==1, "code below assumes"

        # # Get score relative to chance (50%)
        # score_chance = 0.5
        # DFSCORES["score_adjusted"] = (DFSCORES["score"] - score_chance)/(1. - score_chance)

        # ### Plots
        # for do_agg_over_trials in [False, True]:
        #     for morphset_get in [None, "good_ones"]:

        #         dfscores = prune_dfscores_good_morphset(DFSCORES, morphset_get, animal, date)

        #         if do_agg_over_trials:
        #             dfscores = aggregGeneral(dfscores, 
        #                                     ["bregion", "morph_set_idx|idx_within", "idx_within|assigned", "pa_class", "decoder_class", "twind"], 
        #                                     ["score", "score_adjusted"], 
        #                                     nonnumercols=["trial_morph_assigned_to_which_base"])
        #         # (1) 
        #         huevar = "decoder_class_semantic_good"
        #         xvar = "trial_morph_assigned_to_which_base"
        #         xvalues = ["base1", "not_ambig_base1", "ambig_base1", "ambig_base2", "not_ambig_base2", "base2"]
        #         huevalues = ["base1", "base2"]

        #         savedir = f"{SAVEDIR}/do_agg_over_trials={do_agg_over_trials}-morphset_get={morphset_get}/xvar={xvar}"
        #         os.makedirs(savedir, exist_ok=True)

        #         _plot(dfscores, huevar, xvar, xvalues, huevalues, savedir)

        #         # (2)
        #         huevar = "recoded_decoder"
        #         xvar = "recoded_trial_morph"
        #         xvalues = ["ambig", "not_ambig", "base"]
        #         huevalues = ["baseother", "basethis"]

        #         savedir = f"{SAVEDIR}/do_agg_over_trials={do_agg_over_trials}-morphset_get={morphset_get}/xvar={xvar}"
        #         os.makedirs(savedir, exist_ok=True)

        #         _plot(dfscores, huevar, xvar, xvalues, huevalues, savedir)
